In [ ]:
# 有限状态机解决字符整数问题
class FAh2num:
    # 将字符串转换成整数
    def myAtoi(self, s: str) -> int:
        state = [
            {'b':0,'c':1,'d':2,'?':3},# 0
            {'b':3,'c':3,'d':2,'?':3},# 1
            {'b':3,'c':3,'d':2,'?':3}# 2
        ]

        p,t = 0,''
        res,sign = 0,1
        for c in s:
            if '0'<=c<='9':t = 'd'
            elif c == ' ':t = 'b'
            elif c in '+-':t = 'c'
            else:t = '?'

            if t in state[p]:
                p = state[p][t]
                if p == 1:
                    sign = 1 if c == '+' else -1 
                if p == 2:
                    res = res * 10 + ord(c) - ord('0')
                if p == 3:
                    break 
        
        maxNumber,minNumber = 2**31 - 1,-2 ** 31
        
        res *= sign
        if res > maxNumber:res = maxNumber
        if res < minNumber:res = minNumber

        return res

        
    # 表示数值的字符串（有限状态自动机
    def isNumber(self, s: str) -> bool:
        # 状态转移表states，当前状态p，输入字符类型t
        states = [
            { ' ': 0, 's': 1, 'd': 2, '.': 4 }, #0. start with the blank
            { 'd': 2, '.': 4 } ,                # 1. 'sign' before 'e'
            { 'd': 2, '.': 3, 'e': 5, ' ': 8 }, # 2. 'digit' before 'dot'
            { 'd': 3, 'e': 5, ' ': 8 },         # 3. 'digit' after 'dot'
            { 'd': 3 },                         # 4. 'digit' after 'dot' (‘blank’ before 'dot')
            { 's': 6, 'd': 7 },                 # 5. 'e'
            { 'd': 7 },                         # 6. 'sign' after 'e'
            { 'd': 7, ' ': 8 },                 # 7. 'digit' after 'e'
            { ' ': 8 }                          # 8. end with 'blank'
        ]
        p=0                             # start with state 0
        for c in s:
            if '0' <= c <= '9': t = 'd' # digit
            elif c in '+-': t = 's'     # sign
            elif c in 'eE': t = 'e'     # e or E
            elif c in '. ': t = c       # dot, blank
            else: t = '?'               # unknow
            if t not in states[p]: return False
            p = states[p][t]
        return p in (2, 3, 7, 8)

In [ ]:
# 字符匹配KMP算法的next部分匹配表和有限状态机两种解法
class KMPSearch:
    # 暴力匹配思路
    def ViolentMatch(self, s: str, p: str) -> int:
        i, j = 0, 0
        sLen, pLen = len(s), len(p)
        while(i < sLen and j < pLen):
            # 如果j = -1，或者当前字符匹配成功（即S[i] == P[j]），都令i++，j++    
            if s[i] == p[j]:
                i += 1
                j += 1
            else:
                # 如果j != -1，且当前字符匹配失败（即S[i] != P[j]），则令 i 不变，j = next[j], next[j]即为j所对应的next值 
                i = i - j + 1
                j = 0
        if(j == pLen):
            return i-j
        else:
            return -1

    # 递推求next数组
    # 来源：https://blog.csdn.net/v_july_v/article/details/7041827#:~:text=3.3.7-,next%20数组与有限状态自动机
    def GetNext(self,p: str) -> list:
        p, pLen = list(p), len(p)
        j, k = 0, -1
        nList = [-1] * pLen

        while(j < pLen - 1):

            if(k == -1 or p[k] == p[j]):
                k += 1
                j += 1
                nList[j] = k
            else:
                k = nList[k]
        return nList
    
    # 递归求单个next值，来源：https://blog.csdn.net/yyzsir/article/details/89462339
    def GetNext_R(self, j: int, s: list) -> list:
        if j == 0: return -1
        if j > 0:
            k = self.GetNext_R(j-1, s)
            while(k >= 0):
                if s[j - 1] == s[k]: return k + 1
                else: k = self.GetNext_R(k, s)
            return 0
        return 0

    # 执行next版本的kmp搜索
    def kmpMatch(self, s: str, p: str) -> int:
        nList = self.GetNext(p) # 生成模式串p的next数组
        i, j = 0, 0
        sLen, pLen = len(s), len(p)
        while(i < sLen and j < pLen):
            # 如果j = -1，或者当前字符匹配成功（即S[i] == P[j]），都令i++，j++    
            if j == -1 or s[i] == p[j]:
                i += 1
                j += 1
            else:
                # 如果j != -1，且当前字符匹配失败（即S[i] != P[j]），则令 i 不变，j = next[j], next[j]即为j所对应的next值 
                j = nList[j]
        return j == pLen
        # if(j == pLen):
        #     return i-j
        # else:
        #     return -1

    # 有限状态机解法，求解模式p的状态机dfa，来源：https://www.clloz.com/programming/front-end/js/2020/07/24/fsm-kmp/
    def dfaPattern(self, p: str) -> list:
        legalInputs = set(p) # 去重，得到所有可能输入情况
        p, dfa, X = list(p), [], 0 # 重启状态X初始化为0

        dfa.append(dict.fromkeys(legalInputs, 0)) # 初始化dfa[0],即初始的X状态(用字典存储),后面的状态要用这一状态来复制
        dfa[0][p[0]] = 1 # 状态0时匹配到第一位总是进入状态1   

        # 生成后面的状态机
        for j in range(1, len(p)):
            dfa.append(dict(zip(dfa[X].keys(),dfa[X].values()))) # 设置状态j的匹配失败项，从状态X复制，此处实现deepcopy()
            dfa[j][p[j]] = j + 1 # 设置状态j的匹配项
            X = dfa[X][p[j]] # 计算下一状态的重启状态X

        return dfa

    # 有限状态机从状态表匹配主串s
    def dfaMatch(self, s: str, p: str) -> int:
        legalInputs = set(p) # 去重，得到所有可能输入情况
        dfa = self.dfaPattern(p)
        sLen, pLen = len(s), len(p)
        i, j = 0, 0

        while(i < sLen and j < pLen):
            j = dfa[j][s[i]] if s[i] in legalInputs else 0
            i += 1

        return j == pLen